In [114]:
import cantera as ct
import numpy as np
import pandas as pd
import os 
import re
from subprocess import getoutput
import sys
import csv


print("Running Cantera Version: " + str(ct.__version__))

Running Cantera Version: 2.6.0b2


In [115]:
'''copies chemkin files to dups folder '''


os.chdir('/Users/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2F2/chemkin') #put slurm_task id as C2H5F_CH2F2 when I want to generalize the file?
directory ='.'

#list all the input files in the chemkin folder
list_of_inp_files = [file for file in os.listdir(directory) if re.match('chem0([0-9]+)\.inp', file) ]

os.system('source activate ct-env') #if on local, this is cantera 2.6 beta
#os.system('source activate cantera_env') #if on discovery (will be cantera 2.5)

#copy folders so i dont screw up the originals, and change into the new directory with copies
os.makedirs('dups', exist_ok=True)

for file in list_of_inp_files: 
    os.command = f'scp {file} dups/dup_{file}'
    os.system(os.command)
    
dup_files = [file for file in os.listdir('./dups')]
os.system('scp tran.dat ./dups/tran.dat')
os.chdir('./dups') 

   

sh: activate: No such file or directory


In [80]:
'''converts all chemkin files to .cti files'''

#dup_files= ['dup_chem0148.inp'] #test with one file first 

for file in dup_files: 
    x = 0
    while x == 0: 
    #this is a string of the output when I try to convert this file to a cti file, will probably produce an error
        output = getoutput( f'ck2cti --input={file} --transport=tran.dat') 
        #if command passed without an error
        if re.search('PASSED',output):
            print('**************************command passed, converting to cti***************************')
            x += 1 
        #if command generated the Duplicate error
        else:
            print('*******************************needs some work****************************************')
            if re.search('Encountered\sunmarked\sduplicate\sreaction',output):
                match = re.search('See\slines\s([0-9]+)\sand\s([0-9]+)\sof\sthe\sinput\sfile',output)
                #capture the line numbers with the duplicate reactions
                line_numbers = [int(match.group(1)), int(match.group(2))]
                print(f'Unmarked duplicates on lines {match.group(1)} and {match.group(2)}')
                print('Editing chemkin file to allow conversion to .cti')
                #write the lines of the chemkin input file to a list so that I can insert the "DUPLICATE" statement
                with open(file,'r') as f:
                    data = f.readlines()
                    print(data[line_numbers[0]-1], data[line_numbers[1]-1])

                #start editing below

                #'adjustments' will make sure that my, even when I add an element in 'data', my index will still be correct
                adjustments = [0,1]
                for i,adjust in zip(line_numbers,adjustments): 
                    start = i+adjust-1
                    count = 0 
                    while count == 0: 
                        #if you don't see a blank line after the duplicated reaction line, keep going until you do
                        if not re.search('^\\n', data[start]): 
                            print('no match')
                            print(start)
                            print(data[start])
                            start += 1
                        #when we get to the blank line after the reaction block, insert "DUPLICATE" and stop the loop for this line number
                        else: 
                            print('there is a match')
                            data.insert(start,'DUPLICATE')
                            count = 1 
                #now overwrite the file with the change 
                with open(f'{file}','w+') as f: 
                    for l in data: 
                        f.write(l)
                        x==0

            #if the command generated an error that is not the Duplicate error
            else:
                #if code ever gets to here, just cry
                print('There is another error, see Output')
                print(output)
                x += 1

*******************************needs some work****************************************
Unmarked duplicates on lines 992 and 1844
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
991
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
992
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
993
    LOW/ 2.340e+27 -3.180    33.445   /

no match
994
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1844
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1845
    TCHEB/ 300.000   2500.000 /

no match
1846
    PCHEB/ 0.010     98.692   /

no match
1847
    CHEB/ 6 4/

no match
1848
    CHEB/ -2.842e+00   4.139e-01 

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 1244 and 2096
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1243
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
1244
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
1245
    LOW/ 2.340e+27 -3.180    33.445   /

no match
1246
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
2096
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
2097
    TCHEB/ 300.000   2500.000 /

no match
2098
    PCHEB/ 0.010  

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 962 and 1814
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
961
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
962
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
963
    LOW/ 2.340e+27 -3.180    33.445   /

no match
964
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1814
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1815
    TCHEB/ 300.000   2500.000 /

no match
1816
    PCHEB/ 0.010     98

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 1232 and 2084
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1231
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
1232
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
1233
    LOW/ 2.340e+27 -3.180    33.445   /

no match
1234
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
2084
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
2085
    TCHEB/ 300.000   2500.000 /

no match
2086
    PCHEB/ 0.010  

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 710 and 1562
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
709
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
710
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
711
    LOW/ 2.340e+27 -3.180    33.445   /

no match
712
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1562
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1563
    TCHEB/ 300.000   2500.000 /

no match
1564
    PCHEB/ 0.010     98

...

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 872 and 1724
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
871
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
872
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
873
    LOW/ 2.340e+27 -3.180    33.445   /

no match
874
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1724
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1725
    TCHEB/ 300.000   2500.000 /

no match
1726
    PCHEB/ 0.010     98

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 1136 and 1988
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1135
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
1136
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
1137
    LOW/ 2.340e+27 -3.180    33.445   /

no match
1138
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1988
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1989
    TCHEB/ 300.000   2500.000 /

no match
1990
    PCHEB/ 0.010  

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 734 and 1586
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
733
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
734
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
735
    LOW/ 2.340e+27 -3.180    33.445   /

no match
736
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1586
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1587
    TCHEB/ 300.000   2500.000 /

no match
1588
    PCHEB/ 0.010     98

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 1058 and 1910
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1057
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
1058
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
1059
    LOW/ 2.340e+27 -3.180    33.445   /

no match
1060
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1910
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1911
    TCHEB/ 300.000   2500.000 /

no match
1912
    PCHEB/ 0.010  

**************************command passed, converting to cti***************************
*******************************needs some work****************************************
Unmarked duplicates on lines 974 and 1826
Editing chemkin file to allow conversion to .cti
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   
 CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
973
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   2.930e+12 0.290     40.326   

no match
974
H2(8)/2.00/ H2O(9)/6.00/ CO(14)/1.50/ CO2(15)/2.00/ CH4(4)/2.00/ CH2O(21)/2.50/ CH3OH(27)/3.00/ C2H2(24)/3.00/ C2H4(31)/3.00/ C2H6(32)/3.00/ 

no match
975
    LOW/ 2.340e+27 -3.180    33.445   /

no match
976
    TROE/ 2.110e-01 199       2.03e+03  1.12e+05 /

there is a match
no match
1826
CH2CHO(36)(+M)=CO(14)+CH3(20)(+M)                   1.000e+00 0.000     0.000    

no match
1827
    TCHEB/ 300.000   2500.000 /

no match
1828
    PCHEB/ 0.010     98

In [116]:
'''Run this block if you want to see if all inp files were converted to cti files'''


cti_files = [file for file in os.listdir('.') if re.match('dup_chem0([0-9]+)\.cti', file)]

inp_files = [file for file in os.listdir('.') if re.match('dup_chem0([0-9]+)\.inp', file)]


print(f'Number of .cti files: {len(cti_files)} | Number of chemkin files: {len(inp_files)}')

if len(cti_files) == len(inp_files): 
    print('\nAll chemkin files converted')
else: 
    print('\nNot all chemkin files converted')

Number of .cti files: 165 | Number of chemkin files: 165

All chemkin files converted


In [117]:
'''calculates flamespeeds for each .cti files at equivalence ratio = 1. Uses initial guess from the previous model'''

os.chdir('/Users/nora/Code/projects/halogens/refrigerants/blends/C2H5F_CH2F2') #put slurm_task id as C2H5F_CH2F2 when I want to generalize the file?

#cti_files = [file for file in os.listdir('./chemkin/dups') if re.match('dup_chem0([0-9]+)\.cti', file)]

cti_files = ['dup_chem0146.cti']
           
To = 298
Po = 1e5 # ct.one_atm
vol_frac_list = np.arange(0.025, 0.25, step=0.025)

header = ['species']
List_to_write_to_csv = []
for file in cti_files: 
    #make directory to store flame speed calculations, and a header for csv files
    d = f'./flame_calcs_different_models/{file}_CALC/data_richards_py'
    os.makedirs(d, exist_ok=True)
    match1 = re.match('dup_chem0([0-9]+)\.cti',file) 
    gas = ct.Solution(f'./chemkin/dups/{file}')
    results = {}    
    header = ['species']
    #this part calculates the flame calcs for all of the vol_frac list, using the previous vol frac count as a guess for the current one
    for i in  range(len(vol_frac_list)):
        try:
           
            string = f'****************************starting new volume fraction: {vol_frac_list[i]}**************************'
            print(string)
           
            x = vol_frac_list[i]
            norm_ox = (1-x)*.21
            vol_frac_dict = {'C2H5F(1)': (x/2/norm_ox), 
                              'CH2F2(2)': (x/2/norm_ox), 
                              'O2(3)':((1-x)*.21)/norm_ox, 
                              'N2':((1-x)*0.79)/norm_ox } 
            gas.TPX = To, Po, vol_frac_dict
            width = 0.08
            flame = ct.FreeFlame(gas, width=width)
            flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
            flame.max_time_step_count = 900
            loglevel = 1 
            if i!=0:
                d = f'./flame_calcs_different_models/{file}_CALC/data_richards_py/test_{vol_frac_list[i-1]}.csv'
                if os.path.exists(d):  
                    arr2 = ct.SolutionArray(gas)
                    arr2.read_csv(d)
                    flame.set_initial_guess(data=arr2)
                    print(' initial guess has been set')
            #"False" stops the calculation from retrying over and over, thanks Chao 
            flame.solve(loglevel=loglevel, auto=False)
            #flame.solve(loglevel=loglevel, auto=True)
            Su = flame.velocity[0]
            results[x] = Su
            sltn = flame.to_solution_array()
            df1 = sltn.to_pandas()
            df1.to_csv(f'./flame/test_{x}.csv')
            
            

             #delete first column of csv file to avoid error
            df2 = pd.read_csv(f'data_richards/test_{x}.csv')
            # If you know the name of the column skip this
            first_column = df2.columns[0]
            # Delete first
            df2 = df2.drop([first_column], axis=1)
            df2.to_csv(f'./flame_calcs_different_models/{file}_CALC/data_richards_py/test_{x}.csv', index=False)
        except Exception as e: 
            print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
            pass

    vol_fracs = list(results.keys())
    flame_speeds = list(results.values())


    print("volume fractions are:")
    print(vol_fracs)
 
    print("flame speeds are:")
    print(flame_speeds)
    
    header.append(vol_fracs)
    header.append([f'speed_for_volfrac{x}' for x in flame_speeds])
    
    
    values = [vol_fracs, flame_speeds]
    List_to_write_to_csv.append(values)
    

with open('final_calcs.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(List_to_write_to_csv)




****************************starting new volume fraction: 0.025**************************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003844      3.022
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.006568      2.965
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.02494      2.291
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.03551      2.909
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.03371      3.319
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.06399       2.47
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.06074      1.851
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps

Take 10 timesteps     0.0001709      4.405
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.004379      3.313
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01247      2.665
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01184      2.389
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps        0.1011      2.195
Attempt Newton solution of steady-state problem...    success.

Problem solved on [41] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 
    to resolve C#CC(6975) C2H2(24) C2H2F(70) C2H2F2(58) C2H2F3(66) C2H3(30) C2H3F2(63) C2H3F2(64) C2H3F2(65) C2H4(31) C2H4F(61) C2H4F(62) C2H5(33) C2H5F(1) 

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.001095      4.333
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      5.48e-05      5.948
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     5.202e-05      5.885
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0008887      3.595
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01518      2.923
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.05766      4.169
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps        0.9851      2.828
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
######################################################################

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0002563       4.32
Attempt Newton solution of steady-state problem...    success.

Problem solved on [54] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
    to resolve C#CC(6975) C2F(5882) C2H(23) C2H2(24) C2H2F(70) C2H2F2(58) C2H2F3(66) C2H2F3(67) C2H3(30) C2H3F2(63) C2H3F2(64) C2H3F2(65) C2H4(31) C2H4F(61) C2H4F(62) C2H5(33) C2H5F(1) C2H6(32) C2HF(57) C2HF2(71) C2HF4(68) C3H3(6483) CF(50) CF2(43) CF2CH(72) CF2O(49) CFO(51) CFO2(335) CH(18) CH2(S)(26) CH2(T)(19) CH2CF(69) CH2CF2(56) CH2CFO(78) CH2CHF(55) CH2CHO(36) CH2CO(29) CH2F(46) CH2F2(2) CH2O(21) CH2O2(170) CH2OH(34) CH3(20) CH3CHO(37) CH3F(40) CH3O(28) CH3O2(361) CH3OH(27) CH4(4) CHF(41) CHF2(81) CHF3(42) CH

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     5.695e-05      5.727
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.027e-05       5.96
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0007794      4.326
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01332      2.999
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps        0.3413      4.039
Attempt Newton solution of steady-state problem...    success.

Problem solved on [15] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 3 4 5 6 7 8 9 12 
    to resolve C(T)(17) C2F(5882) C2H(23) C2H2(24) C2H2F(70) C2H2F2(58) C2H3(30) C2H4(31) C2H4F(61) C2H4F(62) C2H5F(1) C

...

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0002563      5.119
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.002919      4.174
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.02217      2.933
Attempt Newton solution of steady-state problem...    success.

Problem solved on [30] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
    to resolve C#CC(6975) C2F(5882) C2H(23) C2H2(24) C2H2F(70) C2H2F2(58) C2H2F3(66) C2H2F3(67) C2H3(30) C2H3F2(63) C2H3F2(64) C2H3F2(65) C2H4(31) C2H4F(61) C2H4F(62) C2H5(33) C2H5F(1) C2H6(32) C2HF(57) C2HF(5883) C2HF2(71) C2HF4(68) C3H3(6483) CF(50) CF2(43) CF2CH(72) CF2O(49) CFO(51) CFO2(335) CH2(S)(26) CH2(T

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [184] point grid(s).

..............................................................................
no new points needed in flame
********************passed volume fraction:0.15, error: Cannot save file into a non-existent directory: 'flame'*************************************
****************************starting new volume fraction: 0.17500000000000002**************************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      3.888
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.002919       3.25
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.03325      2.789
Attempt Newton solution of steady-state problem...    

Attempt Newton solution of steady-state problem...    success.

Problem solved on [88] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 59 60 61 
    to resolve C#CC(6975) C2H2F(70) C2H2F2(58) C2H2F3(66) C2H3(30) C2H3F2(63) C2H3F2(64) C2H3F2(65) C2H4(31) C2H4F(61) C2H4F(62) C2H5(33) C2H5F(1) C2H6(32) C2HF(57) C2HF2(71) C2HF4(68) C3H3(6483) CF(50) CF2(43) CF2CH(72) CF2O(49) CFO(51) CFO2(335) CH2CF(69) CH2CF2(56) CH2CFO(78) CH2CHF(55) CH2CHO(36) CH2CO(29) CH2F(46) CH2F2(2) CH2O(21) CH3(20) CH3CHO(37) CH3F(40) CH3OH(27) CH4(4) CHF(41) CHF2(81) CHF3(42) CHFCF2(54) CHFO(47) CO(14) H(5) H2O2(13) HCCO(22) HCO(16) HO2(12) O(6) O2(150) OH(7) S(1097) S(11990) S(12127) S(12564) S(12602) S(13479) S(14526) S

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003844      3.847
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01478      2.871
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps        0.2525      1.781
Attempt Newton solution of steady-state problem...    success.

Problem solved on [51] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 
    to resolve C#CC(6975) C2H2(24) C2H2F(70) C2H2F2(58) C2H2F3(66) C2H2F3(67) C2H3(30) C2H3F2(63) C2H3F2(64) C2H3F2(65) C2H4(31) C2H4F(61) C2H4F(62) C2H5(33) C2H5F(1) C2H6(32) C2HF(57) C2HF2(71) C2HF4(68) C3H3(6483) CF(50) CF2(43) CF2CH(72) CF2O(49) CFO(51) CFO2(335) CH2CF(69) CH2CF2(56)

Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003844      3.867
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.01478      3.101
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps        0.1683      1.737
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps         2.876      1.763
Attempt Newton solution of steady-state problem...    success.

Problem solved on [33] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 
    to resolve C#CC(6975) C2H2(24) C2H2F(70) C2H2F2(58) C2H2F3(66) C2H2F3(67) C2H3(30) C2H3F2(63) C2H3F2(64) C2H3F2(65) C2H4(31) C2H4F(61) C2H4F(62) C2H5(33) C2H5F(1) C2H6(32) C2HF(57) 

In [109]:
import csv
values = (vol_fracs, flame_speeds)
    
    
with open('final_calcs.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(values)
       
        
        
writer.writerow(header)




ValueError: I/O operation on closed file.